# Creating excel file from the 3LE data using the Space-Track.org API

##  This reqiures to login using request and sending a query to the API to fetch the data 

In [10]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd

In [3]:
# What is the login page

url = 'https://www.space-track.org/'
login_route = 'auth/login'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


In [4]:
# Enter the users Space-Track.org username and password manually 

un = input('Enter your username -> ')
pw = input('Enter your password -> ')

Enter your username ->  smallpond.eng.tom@gmail.com
Enter your password ->  svG*kfr6LB5zUyL


In [7]:
# Get the HTML that we want 

with requests.session() as client:
    # Get csrftoken
    cookiejar = client.get(url + login_route).cookies
    cookies = cookiejar.get_dict(domain='www.space-track.org') 
    cookie = cookies['spacetrack_csrf_cookie']  
    
    # What you have to enter to login 
    login_payload = {
        'spacetrack_csrf_token': cookie,
        'identity': un,
        'password': pw,
        'btnLogin': 'LOGIN'
    }
    
    # Post the data to login 
    login_request = client.post(url + login_route, headers=HEADERS, data=login_payload)
    
    # Check if we are logged in or not 
    assert login_request.status_code == 200, 'Not logged in.'  
    cookies = login_request.cookies  # save the cookies for the login 
    
    # Use beautiful soup to extract the HTML that we want 
    api_query = 'https://www.space-track.org/basicspacedata/query/class/gp/EPOCH/%3Enow-30/orderby/NORAD_CAT_ID,EPOCH/format/json'
    TLE_data_response = client.get(api_query) 

In [32]:
# Convert the json data to a csv

TLE_data_json = json.loads(TLE_data_response.text)

In [33]:
# Remove the TLE string lines inside the data that hinders the program from converting the data to an excel file 

# Function to be mapped to delete unnecessary keys inside each dictionary inside the list 
def remove_TLE_keys(data_dict):
    del data_dict['TLE_LINE0']
    del data_dict['TLE_LINE1']
    del data_dict['TLE_LINE2']
    return data_dict

# Remove a certain key insdie the dictionary which is inside a list
TLE_data_json = map(remove_TLE_keys, TLE_data_json)

In [34]:
# Map the operation for all the dictionaries inside the list 
TLE_data_modified_list = list(TLE_data_json)

In [36]:
# Convert the list with dictionaries into a pandas dataframe and convert that to an excel file
df = pd.DataFrame(TLE_data_modified_list)
df.to_excel('TLE_data.xlsx')